In [28]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict
import pandas as pd
import imodelsx.llm
import numpy as np
import paper_parsing
import prompts
import openai
import eval
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()
imodelsx.llm.LLM_CONFIG['LLM_REPEAT_DELAY'] = 10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load df with groundtruth values and paper ids

In [ ]:
# pubmed.download_open_source_papers(df) 
# need to first download papers from https://drive.google.com/drive/folders/1OUXtsddxEAOl3tKEZegBQSwArUecb-6J into ../papers
df, ids_with_paper = paper_parsing.download_gsheet()

### Extract info from the odfs -- add values to the following columns:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

In [ ]:
# extract text from pdfs (create file num.txt for each file num.pdf)
paper_parsing.extract_texts_from_pdf(ids_with_paper, papers_dir=paper_parsing.papers_dir)

# get prompt
llm = imodelsx.llm.get_llm("gpt-4-0613") # gpt-3.5-turbo-0613

# properties, functions, content_str = prompts.get_prompts_gender_and_race()
# print('attempting to add', properties.keys())
# paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

properties, functions, content_str = prompts.get_prompts_gender()
print('attempting to add', properties.keys())
paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

properties, functions, content_str = prompts.get_prompts_race()
print('attempting to add', properties.keys())
paper_parsing.add_columns_based_on_properties(df, ids_with_paper, properties, functions, content_str, llm)

In [22]:
df['num_white'].unique()

array([nan, None, '136', '0', '5345', 'Not mentioned', 'Not specified',
       '92.3', '26003', '348', '662', '248', '80.6%', '2747', '78%',
       'Not provided', 'NA', '33', '87', '27,627', '241', '6655', '778',
       '73 796', '331', '57.4%', '1731', '83', '89.4', '1236', '89.3%',
       'Unknown', '26300', '42%', '513', '71.7', '1429'], dtype=object)

### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [32]:
# # process counts (convert percentages to nums if conditions are correct)
# def process_gender_counts(row):
#     m = row['num_male']
#     f = row['num_female']
#     tot = row['num_total']
#     if eval.str_contains_number(m) and eval.str_is_percentage(m) \
#     and eval.str_contains_number(f) and eval.str_is_percentage(f):
#         m = eval.percentage_to_num(m)
#         f = eval.percentage_to_num(f)
#         if (m + f - 100) < 0.1:
#             print('found perc')
#     return m, f

# df.apply(process_gender_counts, axis=1)

found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc
found perc


0          (55, 50)
1      (None, None)
2        (136, 103)
3      (None, None)
4        (365, 346)
           ...     
685      (nan, nan)
686      (nan, nan)
687      (nan, nan)
688    (2868, 1904)
689      (nan, nan)
Length: 690, dtype: object

In [30]:
d = defaultdict(list)
print('total n', len(ids_with_paper))
for k in df.columns:
    if k.startswith('num_') and k + '_corrected' in df.columns:
        idxs_with_labels = (df[k + '_corrected'].notnull() & ~(df[k + '_corrected'] == 'Unk'))
        gt = df[k + '_corrected'][idxs_with_labels].astype(int)
        pred = df[k].apply(eval.cast_int)[idxs_with_labels].astype(int)
        recall = (np.abs(gt -pred) <= 1).sum()
        d['target'].append(k)
        d['recall'].append(recall)
        d['n_labeled'].append(len(gt))
        # d['n_predicted'].append(df[k].notnull().sum())
        # count number of values which contain a number
        d['n_predicted_num'].append(df[k].apply(eval.str_contains_number).sum())
d = pd.DataFrame.from_dict(d).round(2)
d

total n 184


,target,recall,n_labeled,n_predicted_num
0,num_male,49,69,101
1,num_female,49,69,101
2,num_total,52,69,146
3,num_white,0,0,47
4,num_black,0,0,44
5,num_latino,0,0,33
6,num_asian,0,0,28


In [ ]:
df.to_csv('../data/main.csv', index=False)